# Import dependencies and custom modules

In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'  # Suppress TF log messages

import random
import numpy as np
import tensorflow as tf

In [2]:
from DatasetAPI.DataLoader import DatasetLoader
from Network.BiLSTM_with_Attention import BiLSTM_with_Attention

# Loading the dataset

In [3]:
Model = 'Attention_based_Long_Short_Term_Memory'
DIR = 'DatasetAPI\\EEG-Motor-Movement-Imagery-Dataset\\Processed-Data\\multi_channel'
SAVE = os.path.join('Saved_Files', Model)
os.makedirs(SAVE, exist_ok=True)

# GPU memory growth configuration (TF2 style)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

train_data_list, train_labels_list, test_data_list, test_labels_list = [], [], [], []

for folder in sorted(os.listdir(DIR)):  # Ensure correct ordering
    folder_path = os.path.join(DIR, folder)

    if os.path.isdir(folder_path):  
        
        train_data, train_labels, test_data, test_labels = DatasetLoader(DIR=folder_path)

        # Append to respective lists
        train_data_list.append(train_data)
        train_labels_list.append(train_labels)
        test_data_list.append(test_data)
        test_labels_list.append(test_labels)

train_data = np.vstack(train_data_list)
train_labels = np.vstack(train_labels_list)
test_data = np.vstack(test_data_list)
test_labels = np.vstack(test_labels_list)

# One-hot encode labels (for 4 classes)
train_labels = tf.one_hot(train_labels, depth=4)
train_labels = tf.squeeze(train_labels)
test_labels = tf.one_hot(test_labels, depth=4)
test_labels = tf.squeeze(test_labels)

In [4]:
# Model Hyper-parameters
n_input = 64      # Input size per time step
max_time = 64      # Number of time steps per sequence
lstm_size = 256    # Number of LSTM units (per direction)
attention_size = 8 # Size of the attention layer
n_class = 4        # Number of output classes
n_hidden = 64      # Hidden units in the FC layer
num_epoch = 300    # Number of training epochs
keep_rate = 0.75   # Dropout keep probability

# Learning rate parameters
initial_lr = 1e-4
lr_decay_epoch = 50   # Decay every 50 epochs
lr_decay = 0.50       # Multiply learning rate by 0.5

batch_size = 1024

In [5]:
# Create dataset objects using tf.data
train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_labels))
train_dataset = train_dataset.shuffle(buffer_size=train_data.shape[0]).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((test_data, test_labels))
test_dataset = test_dataset.batch(batch_size)

In [6]:
model = BiLSTM_with_Attention(max_time, n_input, lstm_size, attention_size, 1-keep_rate, n_hidden, n_class)

model.compile(optimizer=tf.keras.optimizers.Adam(initial_lr),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Learning rate scheduling and checkpointing via callbacks
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: initial_lr * (lr_decay ** (epoch // lr_decay_epoch)))
checkpoint = tf.keras.callbacks.ModelCheckpoint(os.path.join(SAVE, "BiLSTM_with_Attention.keras"), monitor='val_accuracy', save_best_only=True)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_layer (ReshapeLayer)    │ (None, 64, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 64, 64, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 64, 64, 10)     │            20 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_1 (Reshape)             │ (None, 64, 640)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 64, 512)        │     1,837,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ attention_layer                 │ [(None, 512), (None,   │             0 │
│ (AttentionLayer)                │ 64, 1)]                │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │        32,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,870,424 (7.14 MB)

 Trainable params: 1,870,296 (7.13 MB)

 Non-trainable params: 128 (512.00 B)

In [7]:
# Training with model.fit handles epoch and batch iterations automatically
history = model.fit(train_dataset, epochs=num_epoch, validation_data=test_dataset, callbacks=[lr_schedule, checkpoint])

Epoch 1/300


81/81 ━━━━━━━━━━━━━━━━━━━━ 1164s 11s/step - accuracy: 0.3073 - loss: 1.3566 - val_accuracy: 0.4740 - val_loss: 1.2494 - learning_rate: 1.0000e-04
Epoch 2/300
81/81 ━━━━━━━━━━━━━━━━━━━━ 1008s 12s/step - accuracy: 0.4763 - loss: 1.0188 - val_accuracy: 0.4953 - val_loss: 1.0515 - learning_rate: 1.0000e-04
Epoch 3/300
81/81 ━━━━━━━━━━━━━━━━━━━━ 1003s 12s/step - accuracy: 0.4878 - loss: 0.9400 - val_accuracy: 0.4991 - val_loss: 0.9409 - learning_rate: 1.0000e-04
Epoch 4/300
81/81 ━━━━━━━━━━━━━━━━━━━━ 1113s 14s/step - accuracy: 0.4950 - loss: 0.9097 - val_accuracy: 0.5042 - val_loss: 0.8994 - learning_rate: 1.0000e-04
Epoch 5/300
81/81 ━━━━━━━━━━━━━━━━━━━━ 877s 11s/step - accuracy: 0.4961 - loss: 0.8942 - val_accuracy: 0.5051 - val_loss: 0.8799 - learning_rate: 1.0000e-04
Epoch 6/300
81/81 ━━━━━━━━━━━━━━━━━━━━ 879s 11s/step - accuracy: 0.4986 - loss: 0.8855 - val_accuracy: 0.5097 - val_loss: 0.8758 - learning_rate: 1.0000e-04
Epoch 7/300
81/81 ━━━━━━━━━━━━━━━━━━━━ 861s 11s/step - accuracy: 0

KeyboardInterrupt: 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from Network.Custom_Layers.ReshapeLayer import ReshapeLayer
from Network.Custom_Layers.AttentionLayer import AttentionLayer

custom_objects = {
    'ReshapeLayer': ReshapeLayer,
    'AttentionLayer': AttentionLayer
}

# model = tf.keras.models.load_model(os.path.join(SAVE, "BiLSTM_with_Attention.keras"), custom_objects=custom_objects)

predictions = model.predict(test_dataset)

predicted_classes = np.argmax(predictions, axis=1) 
true_labels = np.argmax(test_labels, axis=1)

np.savetxt(os.path.join(SAVE, "predictions.csv"), predicted_classes, delimiter=",")
np.savetxt(os.path.join(SAVE, "true_labels.csv"), true_labels, delimiter=",")

accuracy = accuracy_score(true_labels, predicted_classes)
print(f"Test Accuracy: {accuracy:.4f}")

conf_matrix = confusion_matrix(true_labels, predicted_classes)

# Plot Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', 
            xticklabels=np.arange(4), yticklabels=np.arange(4)) 
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

print("\n Classification Report:\n")
print(classification_report(true_labels, predicted_classes))
